In [1]:
%matplotlib inline  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib.animation as animation
import matplotlib
from IPython.display import HTML
import plotly.graph_objects as go  #plotly 4.0.0rc1
from plotly.subplots import make_subplots
from matplotlib import pyplot
import plotly.figure_factory as ff
import os

In [2]:
import itertools
import wandb
print(wandb.__path__)
api = wandb.Api()

from collections import OrderedDict

def get_config(runs):
    config_list = [{k:v for k,v in run.config.items() if not k.startswith('_')} for run in runs]
    return pd.DataFrame.from_records(config_list)

RELEVANT_CFG_KEYS = {
'alpha',
}

def get_history_for_key(runs, key, nan_filler='regret_eval'):
    runs = list(runs)
    history = []
    print(len(runs))
    for count, run in enumerate(runs):
        if count % max(1,int(len(runs)/100)) == 0:
            print(count/len(runs)*100, "%")
        history.append(pd.DataFrame.from_records(run.scan_history(keys=[key])).dropna().reset_index(drop=True))
    config_list = [OrderedDict([(k,str(v)) for k,v in run.config.items() if k in RELEVANT_CFG_KEYS])
                    for run in runs]
    keys = [tuple([run.name] + list(config.values())) for run, config in zip(runs, config_list)]
    names = ['Run'] + list(config_list[0].keys()) + ['LogValue']
    df = pd.concat(history, axis=1, names=names, keys=keys)
    nan_filler = {key: run.summary[nan_filler] for run, key in zip(runs, keys)}
    df = df.fillna(nan_filler)
    return df

['/home/francesco/.local/lib/python3.7/site-packages/wandb']


In [3]:
filter = {
    "sweep": {"$in": ['95gyjb5q']},
#    "config.num_updates": 5,
#    "config.learning_rate": 0.5,
#    "config.noise_policy": {"$in": [1.0, 0.1]},
#    "config.run": 2,
}

runs = api.runs('idsia/renyibounds-multi_armed_bandit', filter)
df = get_history_for_key(runs, 'regret_eval')


120
0.0 %
0.8333333333333334 %
1.6666666666666667 %
2.5 %
3.3333333333333335 %
4.166666666666666 %
5.0 %
5.833333333333333 %
6.666666666666667 %
7.5 %
8.333333333333332 %
9.166666666666666 %
10.0 %
10.833333333333334 %
11.666666666666666 %
12.5 %
13.333333333333334 %
14.166666666666666 %
15.0 %
15.833333333333332 %
16.666666666666664 %
17.5 %
18.333333333333332 %
19.166666666666668 %
20.0 %
20.833333333333336 %
21.666666666666668 %
22.5 %
23.333333333333332 %
24.166666666666668 %
25.0 %
25.833333333333336 %
26.666666666666668 %
27.500000000000004 %
28.333333333333332 %
29.166666666666668 %
30.0 %
30.833333333333336 %
31.666666666666664 %
32.5 %
33.33333333333333 %
34.166666666666664 %
35.0 %
35.833333333333336 %
36.666666666666664 %
37.5 %
38.333333333333336 %
39.166666666666664 %
40.0 %
40.833333333333336 %
41.66666666666667 %
42.5 %
43.333333333333336 %
44.166666666666664 %
45.0 %
45.83333333333333 %
46.666666666666664 %
47.5 %
48.333333333333336 %
49.166666666666664 %
50.0 %
50.8333

/home/francesco/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: PerformanceWarning: indexing past lexsort depth may impact performance.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(df)

Run      chocolate-sweep-120 jumping-sweep-119 pleasant-sweep-118  \
alpha             1000000000        1000000000         1000000000   
LogValue         regret_eval       regret_eval        regret_eval   
0                   5.730026          5.383207           5.176119   
1                   5.730026          5.383207           5.176119   
2                   4.987953          1.334155           4.062613   
3                   0.763169          5.356563           3.764309   
4                   5.638386          4.787456          -0.022901   
..                       ...               ...                ...   
96                  5.819457         -0.403722           5.246062   
97                  5.289516          1.408503           5.154191   
98                  5.806147          0.676007           5.200638   
99                  5.447945          0.264242           4.891200   
100                 5.476829          0.139635           5.250032   

Run      still-sweep-117 volcanic

In [5]:
df = df[1:101]
df = df.transpose()
df

,,,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
Run,alpha,LogValue,,,,,,,,,,,,,,,,,,,,,
chocolate-sweep-120,1000000000,regret_eval,5.730026,4.987953,0.763169,5.638386,5.922722,5.918750,5.815351,1.259607,1.134382,4.652382,...,5.880774,5.908567,5.926736,5.936384,5.850088,5.819457,5.289516,5.806147,5.447945,5.476829
jumping-sweep-119,1000000000,regret_eval,5.383207,1.334155,5.356563,4.787456,3.755877,0.157748,1.175117,-1.021286,0.011546,0.108010,...,0.627374,-1.134827,-0.109167,-0.040137,1.315327,-0.403722,1.408503,0.676007,0.264242,0.139635
pleasant-sweep-118,1000000000,regret_eval,5.176119,4.062613,3.764309,-0.022901,0.198920,-0.972890,4.028104,3.943009,4.593317,4.002514,...,4.656161,5.234842,5.200242,5.281394,5.340480,5.246062,5.154191,5.200638,4.891200,5.250032
still-sweep-117,1000000000,regret_eval,5.045305,-0.183041,0.076756,0.227187,-1.791683,-1.608003,-1.378014,0.104803,4.124778,-0.760139,...,5.123143,5.143176,4.074145,4.985181,4.953332,4.631012,4.523580,4.928055,4.907641,4.777308
volcanic-sweep-116,1000000000,regret_eval,4.868711,3.558346,4.063784,4.019004,4.025417,4.181514,4.094166,4.048045,4.010035,3.898503,...,5.018846,4.973842,4.851789,4.942807,4.900971,3.846671,4.651157,3.973303,5.013383,5.053595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
genial-sweep-1,1e-06,regret_eval,4.854834,-0.077058,3.283769,-0.324193,-0.401015,3.536353,0.989478,-0.744966,4.667444,4.116956,...,4.027227,3.931300,4.056513,4.146749,4.161456,4.044897,3.772542,4.036538,3.963175,3.898632
celestial-sweep-2,1e-06,regret_eval,5.072283,4.169399,5.206980,4.507341,5.051172,3.507199,5.184402,4.268149,4.957389,5.085344,...,-1.971284,-0.003874,0.028932,-0.367114,-0.071738,-0.377228,0.511078,0.167138,-0.810264,0.126492
logical-sweep-5,1e-06,regret_eval,5.097562,4.077491,4.041479,4.255581,4.414508,4.109984,2.294453,4.273596,3.752724,0.327872,...,4.376969,4.276527,4.396727,4.374874,4.308721,4.239070,4.290059,4.255952,4.440464,4.374206


In [6]:
df.to_hdf('data_regret.h5', 'df')